# Segmentation

## Importing dependencies

In [ ]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from skimage import morphology, io, color, exposure, img_as_float, transform
from matplotlib import pyplot as plt
import openslide as op
from PIL import Image
from util import IoU, Dice, masked, remove_small_regions, normalize
from scipy import misc

import time
import h5py

## Configuration

In [ ]:
# Paths
filename = '/root/workspace/data/preExtraction/15_02/IFTA_15_02.svs'
maskname = '/root/workspace/data/preExtraction/15_02/IFTA_15_02.png'
h5f = h5py.File('/root/workspace/data/train/matrice_train.h5','r')

model_name = './model.099.hdf5'

# Do not touch
level=1
patchSize = 256
colorChannels = 3
#stride = patchSize//2
stride = 256

patchShape = (patchSize, patchSize,colorChannels) # Shape of the image (patch)
patchMaskShape = (patchSize, patchSize)   # Shape of the mask (patch)

## Image loading

### Loading base picture

In [ ]:
im = op.OpenSlide(filename)
imload = np.asfarray(im.read_region((0,0), 1, im.level_dimensions[1]),dtype=np.float32)[:,:,0:colorChannels]/255



In [ ]:
#normalization
stats = h5f['stats'][:]
im_mean=stats[0]
im_std=stats[1]
h5f.close()

print(im_mean)
print(im_std)
print(imload.mean())
print(imload.std())

imload-=imload.mean()
imload/=imload.std()
#imload-=im_mean
#imload/=im_std

In [ ]:
print(imload.mean())
print(imload.std())

### Loading mask

In [ ]:
maskload = Image.open(maskname)
maskload = np.expand_dims(np.array(maskload.resize(im.level_dimensions[level]),dtype=np.float32),-1)

### Allocating final array

In [ ]:
outputProba = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0]),dtype=np.float32)
outputProbaTimes = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0]),dtype=np.uint8)

In [ ]:
#outputThreshold = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0], colorChannels),dtype=np.uint8)

## Loading model

In [ ]:
# Load model
UNet = load_model(model_name)

## Execution

### Initialization  

In [ ]:
test_gen = ImageDataGenerator(rescale=1.)

### Prediction

In [ ]:
t0 = time.clock()


# Prediction
for posX in range(0,im.level_dimensions[level][0]-256,stride):
    for posY in range(0,im.level_dimensions[level][1]-256,stride):
        #t0_in = time.clock()
        xx = np.expand_dims(imload[posY:posY+256,posX:posX+256,:],0)
        yy = maskload[posY:posY+256,posX:posX+256,:]

        #xx = np.expand_dims(imload[posY:posY+256,posX:posX+256,:],0)
        #yy = maskload[posY:posY+256,posX:posX+256,:]
        
        #img = exposure.rescale_intensity(np.squeeze(xx), out_range=(0,1))
        
        pred = UNet.predict(xx)[..., 0].reshape(patchShape[:2])
        #mask = yy[..., 0].reshape(patchShape[:2])
        #gt = mask > 0.5
        #pr = pred > 0.5
        #pr = remove_small_regions(pr, 0.02 * np.prod(patchShape))
        
        #outputPart = np.dstack((pr.astype(np.int8), gt.astype(np.int8), pr.astype(np.int8)))

        #    plt.subplot(4, 4, 4 * i + 2)
        #    plt.title('IoU = {:.4f}'.format(ious[i]))
        #    plt.axis('off')
        #    plt.imshow(masked(img, gt, pr, 1))

        part=outputProba[posY:posY+256,posX:posX+256]
        part[:,:]+=pred[0:part.shape[0],0:part.shape[1]]
        outputProbaTimes[posY:posY+256,posX:posX+256]+=1
        
        #t_in = time.clock()
        #print("Execution duration : ", t_in-t0_in)
t = time.clock()
print("Execution duration : ", t-t0)

### code that follows in progress 

In [ ]:
outputProba/=outputProbaTimes

In [ ]:
np.putmask(outputProba, outputProba != outputProba, 0)


In [ ]:
print(outputProba.min())
print(outputProba.max())

In [ ]:
outputProbaThresh = outputProba > 0.5
maskTresh = maskload > 0.5

In [ ]:
outputProbaThresh = remove_small_regions(outputProbaThresh, 0.02 * np.prod(patchShape))

In [ ]:
testDelAmore = np.dstack((outputProbaThresh.astype(np.int8), maskTresh.astype(np.int8), outputProbaThresh.astype(np.int8)))

In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(outputProba,interpolation='none', cmap="jet")
plt.show()

#plt.close()

In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(testDelAmore,interpolation='none', cmap="jet")
plt.show()

In [ ]:
misc.imsave("prediction_15-02_64_new_mean.png", testDelAmore)
